In [3]:
# %pip install python-dotenv transformers accelerate bitsandbytes --quiet # Uncomment if running this file directly and needed.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from dotenv import load_dotenv
import os

c:\Users\Josue\CS-GY-6923 Machine Learning\synap-well\synap-well\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load Environment Variables
load_dotenv()

# Hugging Face Token
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [6]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", token=HUGGING_FACE_TOKEN)

# Load model in 4-bit precision using bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    load_in_4bit=True,
    device_map="auto",
    torch_dtype="auto",
    token=HUGGING_FACE_TOKEN
)

# Create pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample = True,
    pad_token_id=tokenizer.eos_token_id
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [01:11<00:00, 35.52s/it]
Device set to use cuda:0


We decided to opt for "mistralai/Mistral-7B-Instruct-v0.1" due to the following reasons:
1) Very strong instruction following for a 7B model
2) Low VRAM usage (for 7B size) — works on 8–16GB GPUs
3) Very good speed vs. quality balance

As for performance metrics, this model's purpose is for activity recommendation generation, thus other quantitative metrics like BLEU would not be useful. As such, we shall rely on human evaluation as our primary form of quantitive measure.

In [7]:
# Define your prompt and generate activity suggestions function
def generate_activity_suggestions(mood, weather, mental_body_state, preferences):
    prompt = (
        f"[INST] The user is feeling {mood}, the weather is {weather}, and they are currently in a state of {mental_body_state}. "
        f"They typically enjoy activities like {preferences}. "
        f"Based on the weather, user's mood, state, and preferences, suggest 3 relevant activities "
        f"with a 50-word short description for each one, that would produce a healthy mental well-being in the user. [/INST]"
    )
    result = generator(prompt)
    return result[0]["generated_text"]

In [9]:
# Example Input
mood = "happy"
weather = "rainy"
state = "HEALTHY_CONSUMPTION, POOR_SLEEP, GOOD_HEART_HEALTH, GOOD_BlOOD_PRESSURE, ACTIVE, HIGH_Stress, LOW_Workload, HIGH_SOCIAL"
preferences = "socializing, listening to music, playing video games, running, hiking"

output = generate_activity_suggestions(mood, weather, state, preferences)
output = output.split("[/INST]")[1].strip()
print(output)

Activity 1: Rainforest Walk - Take a stroll through an indoor rainforest to get some exercise while also enjoying nature. This activity will help reduce stress levels and improve overall mood.

Activity 2: Indoor Yoga Session - Practice yoga indoors to improve flexibility, strength, and relaxation. It can be done alone or with friends, making it perfect for socialization as well.

Activity 3: Music Therapy Sessions - Attend a music therapy session where you listen to calming and relaxing music to promote relaxation and reduce stress. It can be done individually or in groups, providing opportunities for social interaction.
